# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117085e+02     1.500695e+00
 * time: 0.7661230564117432
     1     1.112615e+01     8.871868e-01
 * time: 2.063620090484619
     2    -1.155866e+01     9.951425e-01
 * time: 2.7383999824523926
     3    -3.394075e+01     7.107279e-01
 * time: 3.6978421211242676
     4    -4.757327e+01     4.999526e-01
 * time: 4.673973083496094
     5    -5.686615e+01     2.152605e-01
 * time: 5.780449151992798
     6    -5.979342e+01     1.151247e-01
 * time: 6.430971145629883
     7    -6.085769e+01     4.902209e-02
 * time: 7.094743967056274
     8    -6.124255e+01     8.768793e-02
 * time: 7.742669105529785
     9    -6.156308e+01     3.051082e-02
 * time: 8.3995840549469
    10    -6.176660e+01     2.903121e-02
 * time: 9.522019147872925
    11    -6.193122e+01     2.142979e-02
 * time: 10.162341117858887
    12    -6.200025e+01     1.772833e-02
 * time: 10.807984113693237
    13    -6.207345e+01     1.581843e-02
 * time: 11.45383906364441
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557690
    AtomicNonlocal      14.8522654
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336821

    total               -62.261666460290
